## LTFS Data Science FinHack 3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# scintific computing libraries
import pandas as pd                                     
import numpy as np                                    
import scipy.optimize as opt                           

# visualisation libraries
import matplotlib.pyplot as plt                                       
import seaborn as sns                                

# algorithmic library
from sklearn import svm                                               
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from xgboost.sklearn import XGBClassifier
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, RandomizedSearchCV, GridSearchCV  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_similarity_score, f1_score, log_loss

%matplotlib inline

### Train_Test dataset

In [3]:
df_train = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LTFS Data Science FinHack 3/train_Data.xlsx')
df_train2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LTFS Data Science FinHack 3/train_bureau.xlsx')

In [67]:
df_test = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LTFS Data Science FinHack 3/test_Data.xlsx')
df_test2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LTFS Data Science FinHack 3/test_bureau.xlsx')

In [5]:
df_train.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [6]:
df_test.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.0,365000.0,1000.0,2011-12-29,2015-10-05,2011-12-29,3524747,1046.0,22354,75.83,M,50.0,32069.00,GUNA,MADHYA PRADESH,473001.0
1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,285000.0,285000.0,9300.0,2012-04-28,2016-01-01,2012-04-28,4985862,1046.0,22354,57.44,M,35.0,25000.00,GUNA,MADHYA PRADESH,473001.0
2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,400000.0,400000.0,35800.0,2013-10-22,2017-09-10,2013-10-22,12881783,1060.0,65929,68.97,M,37.0,23333.33,GUNA,MADHYA PRADESH,473001.0
3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,500000.0,500000.0,52000.0,2013-05-21,2016-02-10,2013-05-21,10613776,1049.0,68597,61.79,M,55.0,91666.67,MAHASAMUND,CHATTISGARH,493558.0
4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,400000.0,400000.0,35400.0,2012-11-30,2016-12-05,2012-11-30,7911110,1049.0,38493,59.09,M,48.0,12500.00,JAGATSINGHAPUR,ORISSA,754137.0


## Data Wrangling

In [7]:
df_train_data = df_train
print(df_train_data.shape)

(128655, 26)


In [8]:
df_train_data['Top-up Month'].value_counts()

No Top-up Service    106677
 > 48 Months           8366
36-48 Months           3656
24-30 Months           3492
30-36 Months           3062
18-24 Months           2368
12-18 Months           1034
Name: Top-up Month, dtype: int64

In [9]:
df_test_data = df_test
print(df_test_data.shape)

(14745, 25)


In [10]:
df_train_data.dtypes

ID                          int64
Frequency                  object
InstlmentMode              object
LoanStatus                 object
PaymentMode                object
BranchID                    int64
Area                       object
Tenure                      int64
AssetCost                   int64
AmountFinance             float64
DisbursalAmount           float64
EMI                       float64
DisbursalDate      datetime64[ns]
MaturityDAte       datetime64[ns]
AuthDate           datetime64[ns]
AssetID                     int64
ManufacturerID              int64
SupplierID                  int64
LTV                       float64
SEX                        object
AGE                       float64
MonthlyIncome             float64
City                       object
State                      object
ZiPCODE                   float64
Top-up Month               object
dtype: object

In [11]:
df_test_data.dtypes

ID                          int64
Frequency                  object
InstlmentMode              object
LoanStatus                 object
PaymentMode                object
BranchID                    int64
Area                       object
Tenure                      int64
AssetCost                   int64
AmountFinance             float64
DisbursalAmount           float64
EMI                       float64
DisbursalDate      datetime64[ns]
MaturityDAte       datetime64[ns]
AuthDate           datetime64[ns]
AssetID                     int64
ManufacturerID            float64
SupplierID                  int64
LTV                       float64
SEX                        object
AGE                       float64
MonthlyIncome             float64
City                       object
State                      object
ZiPCODE                   float64
dtype: object

In [12]:
df_train_data.isnull().sum()

ID                     0
Frequency              0
InstlmentMode          0
LoanStatus             0
PaymentMode            0
BranchID               0
Area               11653
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
EMI                    0
DisbursalDate          0
MaturityDAte           1
AuthDate               0
AssetID                0
ManufacturerID         0
SupplierID             0
LTV                    0
SEX                   59
AGE                   59
MonthlyIncome        234
City               11256
State                  0
ZiPCODE              372
Top-up Month           0
dtype: int64

In [13]:
df_test_data.isnull().sum()

ID                   0
Frequency            0
InstlmentMode        0
LoanStatus           0
PaymentMode          0
BranchID             0
Area               546
Tenure               0
AssetCost            0
AmountFinance        0
DisbursalAmount      0
EMI                  0
DisbursalDate        0
MaturityDAte         0
AuthDate             0
AssetID              0
ManufacturerID       2
SupplierID           0
LTV                  0
SEX                  5
AGE                  7
MonthlyIncome       14
City                 0
State                0
ZiPCODE              1
dtype: int64

In [14]:
col_drop = ['AssetID','Area','City','ZiPCODE']
df_train_data.drop(col_drop, axis=1, inplace=True)
df_test_data.drop(col_drop, axis=1, inplace=True)

In [15]:
df_train_data['SEX'].replace(np.nan, 'M', inplace = True)
df_test_data['SEX'].replace(np.nan, 'M', inplace = True)
col_replace = ['AGE','MonthlyIncome','ManufacturerID']
for col in col_replace:
  df_train_data[col].replace(np.nan, df_train_data[col].median(), inplace = True)
  df_test_data[col].replace(np.nan, df_train_data[col].median(), inplace = True)

In [16]:
col_dropna = ['MaturityDAte']
df_train_data.dropna(subset = col_dropna, axis = 0, inplace = True)

In [17]:
col_label_encoder = ['Frequency','InstlmentMode','LoanStatus','EMI','ManufacturerID','SupplierID','SEX','State']
for col in col_label_encoder:
    df_train_data[col]= LabelEncoder().fit_transform(df_train_data[col])
    df_test_data[col]= LabelEncoder().fit_transform(df_test_data[col])

In [18]:
df_train_data['PaymentMode'].unique()

array(['PDC_E', 'PDC', 'Direct Debit', 'Billed', 'ECS', 'Auto Debit',
       'SI Reject', 'ECS Reject', 'Cheque', 'PDC Reject', 'Escrow'],
      dtype=object)

In [19]:
d = {'Auto Debit':0,'Direct Debit':1,'Billed':2,'Escrow':3,'SI Reject':4,'ECS':5,'ECS Reject':5,'PDC_E':6,'PDC':6,'PDC Reject':6,'Cheque':6}
l = list(df_train_data['PaymentMode'])
for i in range(len(l)):
  l[i] = d[l[i]]
df_train_data['PaymentMode'] = l

l2 = list(df_test_data['PaymentMode'])
for i in range(len(l2)):
  l2[i] = d[l2[i]]
df_test_data['PaymentMode'] = l2

In [ ]:
col_date = ['DisbursalDate','MaturityDAte','AuthDate']

In [ ]:
'''l = []
for i in range(df_train_data.shape[0]):
    try:
        x = ((df_train_data['MaturityDAte'][i].year - df_train_data['DisbursalDate'][i].year)*12) + (df_train_data['MaturityDAte'][i].month - df_train_data['DisbursalDate'][i].month)
        l.append(x)
    except:
        l.append(0)
df_train_data['MaturityDAte_DisbursalDate'] = l'''

In [ ]:
'''l = []
for i in range(df_test_data.shape[0]):
    try:
        x = ((df_test_data['MaturityDAte'][i].year - df_test_data['DisbursalDate'][i].year)*12) + (df_test_data['MaturityDAte'][i].month - df_test_data['DisbursalDate'][i].month)
        l.append(x)
    except:
        l.append(0)
df_test_data['MaturityDAte_DisbursalDate'] = l'''

In [ ]:
'''df_train_data.drop(col_date, axis=1, inplace=True)
df_test_data.drop(col_date, axis=1, inplace=True)'''

In [20]:
print(df_train_data.shape)
df_train_data.head()

(128654, 22)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State,Top-up Month
0,1,2,1,1,6,1,48,450000,275000.0,275000.0,3745,2012-02-10,2016-01-15,2012-02-10,6,110,61.11,1,49.0,35833.33,12,> 48 Months
1,2,2,0,1,6,333,47,485000,350000.0,350000.0,943,2012-03-31,2016-02-15,2012-03-31,3,928,70.00,1,23.0,666.67,12,No Top-up Service
2,3,3,1,0,1,1,68,690000,519728.0,519728.0,7174,2017-06-17,2023-02-10,2017-06-17,2,3560,69.77,1,39.0,45257.00,12,12-18 Months
3,7,2,0,1,2,125,48,480000,400000.0,400000.0,1316,2013-11-29,2017-11-10,2013-11-29,2,585,80.92,1,24.0,20833.33,12,> 48 Months
4,8,2,1,1,2,152,44,619265,440000.0,440000.0,2493,2011-12-08,2015-07-05,2011-12-08,1,104,71.05,1,56.0,27313.67,4,36-48 Months


In [21]:
print(df_test_data.shape)
df_test_data.head()

(14745, 21)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State
0,4,1,0,1,6,2,46,480000,365000.0,365000.0,5,2011-12-29,2015-10-05,2011-12-29,1,68,75.83,1,50.0,32069.00,9
1,5,1,0,1,6,2,45,480000,285000.0,285000.0,140,2012-04-28,2016-01-01,2012-04-28,1,68,57.44,1,35.0,25000.00,9
2,6,2,1,1,6,2,48,580000,400000.0,400000.0,1092,2013-10-22,2017-09-10,2013-10-22,3,710,68.97,1,37.0,23333.33,9
3,25,0,1,1,2,154,36,725000,500000.0,500000.0,1700,2013-05-21,2016-02-10,2013-05-21,2,731,61.79,1,55.0,91666.67,2
4,119,2,1,1,6,194,48,617000,400000.0,400000.0,1073,2012-11-30,2016-12-05,2012-11-30,2,485,59.09,1,48.0,12500.00,11


### Bureau Data

In [22]:
df_train2.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [68]:
df_test2.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,NaN,589,NaN,0,0.0,NaN,"20191117,",000,"589,","0,","0,",NaN
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,"25,00,000","30,797/Monthly","21,07,227",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,"99,000","1,223/Monthly","83,458",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,"3,65,000",NaN,0,NaN,0,0.0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,"25,801",NaN,0,NaN,0,0.0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",NaN


In [69]:
df_train_bureau = df_train2
print(df_train_bureau.shape)

(507265, 15)


In [70]:
df_test_bureau = df_test2
print(df_test_bureau.shape)

(64019, 25)


In [71]:
df_train_bureau.dtypes

ID                                    int64
SELF-INDICATOR                        int64
MATCH-TYPE                            int64
ACCT-TYPE                             int64
CONTRIBUTOR-TYPE                      int64
DATE-REPORTED                datetime64[ns]
OWNERSHIP-IND                         int64
ACCOUNT-STATUS                        int64
DISBURSED-DT                 datetime64[ns]
DISBURSED-AMT/HIGH CREDIT           float64
CURRENT-BAL                         float64
WRITE-OFF-AMT                       float64
CUR BAL - HIST                        int64
AMT OVERDUE - HIST                    int64
AMT PAID - HIST                       int64
dtype: object

In [72]:
df_test_bureau.dtypes

ID                                    int64
SELF-INDICATOR                         bool
MATCH-TYPE                           object
ACCT-TYPE                            object
CONTRIBUTOR-TYPE                     object
DATE-REPORTED                datetime64[ns]
OWNERSHIP-IND                        object
ACCOUNT-STATUS                       object
DISBURSED-DT                 datetime64[ns]
CLOSE-DT                             object
LAST-PAYMENT-DATE            datetime64[ns]
CREDIT-LIMIT/SANC AMT                object
DISBURSED-AMT/HIGH CREDIT            object
INSTALLMENT-AMT                      object
CURRENT-BAL                          object
INSTALLMENT-FREQUENCY                object
OVERDUE-AMT                          object
WRITE-OFF-AMT                       float64
ASSET_CLASS                          object
REPORTED DATE - HIST                 object
DPD - HIST                           object
CUR BAL - HIST                       object
AMT OVERDUE - HIST              

In [50]:
df_train_bureau.isnull().sum()

ID                           0
SELF-INDICATOR               0
MATCH-TYPE                   0
ACCT-TYPE                    0
CONTRIBUTOR-TYPE             0
DATE-REPORTED                0
OWNERSHIP-IND                0
ACCOUNT-STATUS               0
DISBURSED-DT                 0
DISBURSED-AMT/HIGH CREDIT    0
CURRENT-BAL                  0
WRITE-OFF-AMT                0
CUR BAL - HIST               0
AMT OVERDUE - HIST           0
AMT PAID - HIST              0
dtype: int64

In [51]:
df_test_bureau.isnull().sum()

ID                               0
SELF-INDICATOR                   0
MATCH-TYPE                       0
ACCT-TYPE                        0
CONTRIBUTOR-TYPE                 0
DATE-REPORTED                    0
OWNERSHIP-IND                    0
ACCOUNT-STATUS                   0
DISBURSED-DT                    44
CLOSE-DT                     31453
LAST-PAYMENT-DATE            32550
CREDIT-LIMIT/SANC AMT        62421
DISBURSED-AMT/HIGH CREDIT       38
INSTALLMENT-AMT              47693
CURRENT-BAL                     32
INSTALLMENT-FREQUENCY        48243
OVERDUE-AMT                  13127
WRITE-OFF-AMT                 2561
ASSET_CLASS                  34902
REPORTED DATE - HIST          2561
DPD - HIST                    2565
CUR BAL - HIST                2561
AMT OVERDUE - HIST            2561
AMT PAID - HIST               2672
TENURE                       42882
dtype: int64

In [73]:
col_drop = ['CLOSE-DT','LAST-PAYMENT-DATE','CREDIT-LIMIT/SANC AMT','INSTALLMENT-AMT','INSTALLMENT-FREQUENCY',
               'OVERDUE-AMT','ASSET_CLASS','TENURE','DPD - HIST','REPORTED DATE - HIST']
df_train_bureau.drop(col_drop, axis=1, inplace=True)
df_test_bureau.drop(col_drop, axis=1, inplace=True)

In [74]:
col_dropna = ['DATE-REPORTED','DISBURSED-DT','CURRENT-BAL','WRITE-OFF-AMT','CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST','DISBURSED-AMT/HIGH CREDIT']
df_train_bureau.dropna(subset = col_dropna, axis = 0, inplace = True)
df_test_bureau.dropna(subset = col_dropna, axis = 0, inplace = True)

In [75]:
col_date = ['DATE-REPORTED','DISBURSED-DT']

In [ ]:
'''l = []
for i in range(df_train_bureau.shape[0]):
    try:
        x = ((df_train_bureau['DATE-REPORTED'][i].year - df_train_bureau['DISBURSED-DT'][i].year)*12) + (df_train_bureau['DATE-REPORTED'][i].month - df_train_bureau['DISBURSED-DT'][i].month)
        l.append(x)
    except:
        l.append(0)
df_train_bureau['DATE-REPORTED_DISBURSED-DT'] = l'''

In [ ]:
'''l = []
for i in range(df_test_bureau.shape[0]):
    try:
        x = ((df_test_bureau['DATE-REPORTED'][i].year - df_test_bureau['DISBURSED-DT'][i].year)*12) + (df_test_bureau['DATE-REPORTED'][i].month - df_test_bureau['DISBURSED-DT'][i].month)
        l.append(x)
    except:
        l.append(0)
df_test_bureau['DATE-REPORTED_DISBURSED-DT'] = l'''

In [ ]:
'''df_train_bureau.drop(col_date, axis=1, inplace=True)
df_test_bureau.drop(col_date, axis=1, inplace=True)'''

In [76]:
col_label_encoder = ['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS']
for col in col_label_encoder:
    df_train_bureau[col]= LabelEncoder().fit_transform(df_train_bureau[col])
    df_test_bureau[col]= LabelEncoder().fit_transform(df_test_bureau[col])

In [77]:
column = ['DISBURSED-AMT/HIGH CREDIT','CURRENT-BAL']
for col in column:
  df_train_bureau[col] = df_train_bureau[col].str.replace(',','').astype(float)
  df_test_bureau[col] = df_test_bureau[col].str.replace(',','').astype(float)

In [34]:
column = ['CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
for col in column:
    l = list(df_train_bureau[col])
    for i in range(len(l)):
      try:
        l2 = l[i].split(',')
        l3 = []
        for x in l2:
            try:
                l3.append(int(x))
            except:
                continue
                
        try:
            l[i] = sum(l3)//len(l3)
        except:
            l[i] = 0
      except:
        l[i] = 0
    df_train_bureau[col] = l

In [78]:
column = ['CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
for col in column:
    l = list(df_test_bureau[col])
    for i in range(len(l)):
      try:
        l2 = l[i].split(',')
        l3 = []
        for x in l2:
            try:
                l3.append(int(x))
            except:
                continue
                
        try:
            l[i] = sum(l3)//len(l3)
        except:
            l[i] = 0
      except:
        l[i] = 0
    df_test_bureau[col] = l

In [79]:
df_train_bureau = df_train_bureau.drop_duplicates(['ID','DISBURSED-AMT/HIGH CREDIT','DISBURSED-DT'], keep='last')
df_test_bureau = df_test_bureau.drop_duplicates(['ID','DISBURSED-AMT/HIGH CREDIT','DISBURSED-DT'], keep='last')

In [101]:
df_merge_train = pd.merge(df_train_data, df_train_bureau, how='left', left_on=['ID','DisbursalAmount','DisbursalDate'], right_on=['ID','DISBURSED-AMT/HIGH CREDIT','DISBURSED-DT'])

In [102]:
df_merge_test = pd.merge(df_test_data, df_test_bureau, how='left', left_on=['ID','DisbursalAmount','DisbursalDate'], right_on=['ID','DISBURSED-AMT/HIGH CREDIT','DISBURSED-DT'])

In [41]:
print(df_merge_train.shape)
df_merge_train.head()

(128654, 36)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State,Top-up Month,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,DISBURSED-AMT/HIGH CREDIT,CURRENT-BAL,WRITE-OFF-AMT,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST
0,1,2,1,1,6,1,48,450000,275000.0,275000.0,3745,2012-02-10,2016-01-15,2012-02-10,6,110,61.11,1,49.0,35833.33,12,> 48 Months,1.0,0.0,46.0,7.0,2016-02-29,1.0,1.0,2012-02-10,275000.0,0.0,0.0,118176.0,0.0,0.0
1,2,2,0,1,6,333,47,485000,350000.0,350000.0,943,2012-03-31,2016-02-15,2012-03-31,3,928,70.00,1,23.0,666.67,12,No Top-up Service,1.0,0.0,46.0,7.0,2016-02-29,1.0,1.0,2012-03-31,350000.0,0.0,0.0,149432.0,1740.0,0.0
2,3,3,1,0,1,1,68,690000,519728.0,519728.0,7174,2017-06-17,2023-02-10,2017-06-17,2,3560,69.77,1,39.0,45257.00,12,12-18 Months,1.0,0.0,46.0,7.0,2020-01-31,1.0,0.0,2017-06-17,519728.0,37637.0,0.0,413400.0,2590.0,0.0
3,7,2,0,1,2,125,48,480000,400000.0,400000.0,1316,2013-11-29,2017-11-10,2013-11-29,2,585,80.92,1,24.0,20833.33,12,> 48 Months,1.0,0.0,46.0,7.0,2017-11-30,1.0,1.0,2013-11-29,400000.0,0.0,0.0,168979.0,0.0,0.0
4,8,2,1,1,2,152,44,619265,440000.0,440000.0,2493,2011-12-08,2015-07-05,2011-12-08,1,104,71.05,1,56.0,27313.67,4,36-48 Months,1.0,0.0,46.0,7.0,2015-07-31,1.0,1.0,2011-12-08,440000.0,0.0,0.0,243891.0,0.0,0.0


In [163]:
df_merge_train.isnull().sum()

ID                            0
Frequency                     0
InstlmentMode                 0
LoanStatus                    0
PaymentMode                   0
BranchID                      0
Tenure                        0
AssetCost                     0
AmountFinance                 0
DisbursalAmount               0
EMI                           0
ManufacturerID                0
SupplierID                    0
LTV                           0
SEX                           0
AGE                           0
MonthlyIncome                 0
State                         0
Top-up Month                  0
SELF-INDICATOR                0
MATCH-TYPE                    0
ACCT-TYPE                     0
CONTRIBUTOR-TYPE              0
OWNERSHIP-IND                 0
ACCOUNT-STATUS                0
CURRENT-BAL                   0
WRITE-OFF-AMT                 0
CUR BAL - HIST                0
AMT OVERDUE - HIST            0
AMT PAID - HIST               0
MaturityDAte_DisbursalDate    0
dtype: i

In [81]:
print(df_merge_test.shape)
df_merge_test.head()

(14745, 35)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,DISBURSED-AMT/HIGH CREDIT,CURRENT-BAL,WRITE-OFF-AMT,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST
0,4,1,0,1,6,2,46,480000,365000.0,365000.0,5,2011-12-29,2015-10-05,2011-12-29,1,68,75.83,1,50.0,32069.00,9,1.0,0.0,41.0,6.0,2015-10-31,1.0,1.0,2011-12-29,365000.0,0.0,0.0,167493.0,314.0,0.0
1,5,1,0,1,6,2,45,480000,285000.0,285000.0,140,2012-04-28,2016-01-01,2012-04-28,1,68,57.44,1,35.0,25000.00,9,1.0,0.0,41.0,6.0,2014-12-31,1.0,1.0,2012-04-28,285000.0,0.0,0.0,185519.0,696.0,0.0
2,6,2,1,1,6,2,48,580000,400000.0,400000.0,1092,2013-10-22,2017-09-10,2013-10-22,3,710,68.97,1,37.0,23333.33,9,1.0,0.0,41.0,6.0,2017-10-31,1.0,1.0,2013-10-22,400000.0,0.0,0.0,167971.0,1058.0,0.0
3,25,0,1,1,2,154,36,725000,500000.0,500000.0,1700,2013-05-21,2016-02-10,2013-05-21,2,731,61.79,1,55.0,91666.67,2,1.0,0.0,41.0,6.0,2016-02-29,1.0,1.0,2013-05-21,500000.0,0.0,0.0,259171.0,0.0,0.0
4,119,2,1,1,6,194,48,617000,400000.0,400000.0,1073,2012-11-30,2016-12-05,2012-11-30,2,485,59.09,1,48.0,12500.00,11,1.0,0.0,41.0,6.0,2017-04-30,1.0,1.0,2012-11-30,400000.0,0.0,0.0,147201.0,317.0,0.0


In [162]:
df_merge_test.isnull().sum()

ID                            0
Frequency                     0
InstlmentMode                 0
LoanStatus                    0
PaymentMode                   0
BranchID                      0
Tenure                        0
AssetCost                     0
AmountFinance                 0
DisbursalAmount               0
EMI                           0
ManufacturerID                0
SupplierID                    0
LTV                           0
SEX                           0
AGE                           0
MonthlyIncome                 0
State                         0
SELF-INDICATOR                0
MATCH-TYPE                    0
ACCT-TYPE                     0
CONTRIBUTOR-TYPE              0
OWNERSHIP-IND                 0
ACCOUNT-STATUS                0
CURRENT-BAL                   0
WRITE-OFF-AMT                 0
CUR BAL - HIST                0
AMT OVERDUE - HIST            0
AMT PAID - HIST               0
MaturityDAte_DisbursalDate    0
dtype: int64

In [107]:
l = []
for i in range(df_merge_train.shape[0]):
    try:
        x = ((df_merge_train['MaturityDAte'][i].year - df_merge_train['DisbursalDate'][i].year)*12) + (df_merge_train['MaturityDAte'][i].month - df_merge_train['DisbursalDate'][i].month)
        l.append(x)
    except:
        l.append(0)
df_merge_train['MaturityDAte_DisbursalDate'] = l

In [108]:
l = []
for i in range(df_merge_test.shape[0]):
    try:
        x = ((df_merge_test['MaturityDAte'][i].year - df_merge_test['DisbursalDate'][i].year)*12) + (df_merge_test['MaturityDAte'][i].month - df_merge_test['DisbursalDate'][i].month)
        l.append(x)
    except:
        l.append(0)
df_merge_test['MaturityDAte_DisbursalDate'] = l

In [109]:
col_drop = ['DISBURSED-DT','DISBURSED-AMT/HIGH CREDIT','DisbursalDate','MaturityDAte','AuthDate','DATE-REPORTED']
df_merge_train.drop(col_drop, axis=1, inplace=True)
df_merge_test.drop(col_drop, axis=1, inplace=True)

In [105]:
col_label_encoder = ['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS']
for col in col_label_encoder:
    df_merge_train[col]= LabelEncoder().fit_transform(df_merge_train[col])
    df_merge_test[col]= LabelEncoder().fit_transform(df_merge_test[col])

In [110]:
col_replace = ['CURRENT-BAL','WRITE-OFF-AMT','CUR BAL - HIST','AMT OVERDUE - HIST','AMT PAID - HIST']
for col in col_replace:
  df_merge_train[col].replace(np.nan, df_merge_train[col].median(), inplace = True)
  df_merge_test[col].replace(np.nan, df_merge_test[col].median(), inplace = True)

In [ ]:
'''from collections import Counter
col_categories = ['ID','SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE','OWNERSHIP-IND', 'ACCOUNT-STATUS']
d = dict()
for col in col_categories:
  a1 = list(df2['ID'])
  a2 = list(df2[col])
  x = a1[0]
  a3 = []
  a4 = []
  for i in range(len(a1)):
    if a1[i] == x:
      a3.append(a2[i])
    else:
      data = Counter(a3)
      y = data.most_common(1)
      a4.append(y[0][0])
      x = a1[i]
      a3.clear()
      a3.append(a2[i])
  data = Counter(a3)
  y = data.most_common(1)
  a4.append(y[0][0])
  d[col] = a4

In [ ]:
'''df2_test_groupby = pd.DataFrame(d)
print(df2_test_groupby.shape)
df2_test_groupby.head()

In [ ]:
'''df_test_groupby = pd.merge(df1_test_groupby, df2_test_groupby, how='left', on=['ID'])
print(df_test_groupby.shape)
df_test_groupby.head()

In [ ]:
'''df_train_groupby = df_train_bureau.groupby(['ID']).mean()
ID = df_train_groupby.index
df_train_groupby['ID'] = ID 
df_train_groupby = df_train_groupby.reset_index(drop=True)
print(df_train_groupby.shape)
df_train_groupby.head()'''

In [ ]:
'''l = []
for i in range(df_merge.shape[0]):
    try:
        if df_merge['DISBURSED-DT'][i].year < df_merge['MaturityDAte'][i].year:
            l.append(0)
        elif df_merge['DISBURSED-DT'][i].year > df_merge['MaturityDAte'][i].year:
            l.append(1)
        else:
            if df_merge['DISBURSED-DT'][i].month < df_merge['MaturityDAte'][i].month:
                l.append(0)
            elif df_merge['DISBURSED-DT'][i].month > df_merge['MaturityDAte'][i].month:
                l.append(1)
            else:
                l.append(0)
    except:
        l.append(0)
df_merge['MaturityDAte_DISBURSED-DT'] = l'''

In [ ]:
'''for col in col_categories:
  l = pd.cut(df_merge[col],100,labels=range(100))
  df_merge[col] = l
  print(col,df_merge[col].unique())'''

In [ ]:
'''col_replace = ['SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
       'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'DISBURSED-AMT/HIGH CREDIT',
       'CURRENT-BAL', 'WRITE-OFF-AMT', 'CUR BAL - HIST', 'AMT OVERDUE - HIST',
       'AMT PAID - HIST', 'DATE-REPORTED_DISBURSED-DT']
for col in col_replace:
  df_merge[col].replace(np.nan, df_merge[col].median(), inplace = True)'''

In [ ]:
'''df_test_groupby = df_test_bureau.groupby(['ID']).mean()
ID = df_test_groupby.index
df_test_groupby['ID'] = ID 
df_test_groupby = df_test_groupby.reset_index(drop=True)
print(df_test_groupby.shape)
df_test_groupby.head()'''

In [112]:
df_merge_train['Top-up Month'].value_counts()

No Top-up Service    107030
 > 48 Months           8398
36-48 Months           3701
24-30 Months           3507
30-36 Months           3075
18-24 Months           2374
12-18 Months           1043
Name: Top-up Month, dtype: int64

In [113]:
l = list(df_merge_train['Top-up Month'])
d = {'No Top-up Service':0,'12-18 Months':1,'18-24 Months':2,'24-30 Months':3,'30-36 Months':4,'36-48 Months':5,' > 48 Months':6}
for i in range(len(l)):
  l[i] = d[l[i]]
df_merge_train['Top-up Month'] = l

## Data Exploration

In [132]:
df_merge_train.corr()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State,Top-up Month,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,CURRENT-BAL,WRITE-OFF-AMT,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,MaturityDAte_DisbursalDate
ID,1.000000,-0.092425,0.111757,-0.240009,0.004595,0.355969,-0.038136,0.067638,0.101560,0.101462,0.051830,0.137881,0.314407,0.072339,-0.027966,-0.053636,0.006439,0.012087,0.019500,0.069260,0.069260,0.056443,0.069291,0.075525,0.058637,0.176729,NaN,0.156935,0.009805,NaN,0.039923
Frequency,-0.092425,1.000000,-0.050171,0.042971,-0.149099,-0.066303,-0.161446,-0.056901,-0.183339,-0.184642,-0.515236,-0.014930,0.021127,-0.112925,-0.001074,-0.071440,0.002266,0.242329,-0.039265,-0.009865,-0.009865,-0.028166,-0.009904,-0.006590,-0.019792,-0.064147,NaN,-0.123573,-0.014175,NaN,-0.134593
InstlmentMode,0.111757,-0.050171,1.000000,0.000679,0.091690,0.096566,0.023031,-0.031050,0.023064,0.024596,0.334230,0.044478,0.040983,0.030673,0.001392,0.029809,0.001519,-0.063748,0.002618,-0.039291,-0.039291,-0.043017,-0.039302,-0.036955,-0.048515,0.017667,NaN,0.054936,0.020136,NaN,0.049690
LoanStatus,-0.240009,0.042971,0.000679,1.000000,0.117193,-0.068237,-0.116837,-0.136786,-0.229022,-0.225215,-0.001817,0.039913,-0.282642,-0.156558,0.049483,0.056994,-0.006711,-0.047113,-0.059019,-0.395045,-0.395045,-0.395200,-0.395061,-0.396564,-0.396188,-0.476964,NaN,-0.201360,0.070577,NaN,-0.279794
PaymentMode,0.004595,-0.149099,0.091690,0.117193,1.000000,0.014835,-0.007302,-0.033859,-0.053474,-0.052517,0.144041,0.020513,-0.079748,-0.057718,0.012234,0.008809,0.001753,-0.016021,0.011572,-0.083391,-0.083391,-0.078155,-0.083391,-0.083594,-0.079175,-0.092304,NaN,-0.086100,0.034618,NaN,-0.097860
BranchID,0.355969,-0.066303,0.096566,-0.068237,0.014835,1.000000,0.064253,-0.032007,0.033926,0.033252,0.008344,-0.004455,0.162154,0.074397,0.029062,-0.004228,0.002884,0.062447,-0.013476,0.024863,0.024863,-0.007720,0.024871,0.032459,-0.011733,0.051366,NaN,0.016090,-0.004152,NaN,0.056373
Tenure,-0.038136,-0.161446,0.023031,-0.116837,-0.007302,0.064253,1.000000,0.067327,0.366051,0.364451,0.055386,0.006309,-0.056543,0.416375,0.009799,0.009434,-0.000486,-0.023496,0.034245,0.013754,0.013754,0.023781,0.013761,0.013745,0.022059,0.122778,NaN,0.200437,0.089858,NaN,0.612756
AssetCost,0.067638,-0.056901,-0.031050,-0.136786,-0.033859,-0.032007,0.067327,1.000000,0.597446,0.597236,0.261233,-0.121853,0.043438,-0.043036,-0.037321,-0.038395,0.003322,-0.008832,0.052199,0.110551,0.110551,0.097866,0.110558,0.107514,0.098648,0.147496,NaN,0.395308,0.064759,NaN,0.144534
AmountFinance,0.101560,-0.183339,0.023064,-0.229022,-0.053474,0.033926,0.366051,0.597446,1.000000,0.998736,0.375805,-0.044543,0.008941,0.708573,-0.038241,-0.030527,0.002331,-0.104272,0.100125,0.087022,0.087022,0.087905,0.087040,0.084148,0.087063,0.272285,NaN,0.642827,0.170439,NaN,0.610022
DisbursalAmount,0.101462,-0.184642,0.024596,-0.225215,-0.052517,0.033252,0.364451,0.597236,0.998736,1.000000,0.380183,-0.043867,0.006731,0.708813,-0.038097,-0.029721,0.002353,-0.106197,0.100400,0.083366,0.083366,0.084628,0.083385,0.080503,0.083691,0.271031,NaN,0.642953,0.171694,NaN,0.607951


In [160]:
from scipy import stats
for col in df_merge_train.columns:
  pearson_coef, p_value = stats.pearsonr(df_merge_train[col], df_merge_train['Top-up Month'])
  print(col, ':', pearson_coef, ',', p_value)  

ID : 0.019500354006641345 , 2.4186859210668205e-12
Frequency : -0.03926539603312569 , 3.066465682448583e-45
InstlmentMode : 0.002617900967021093 , 0.34684882404107065
LoanStatus : -0.05901862937715513 , 5.462015752975159e-100
PaymentMode : 0.011572493686162998 , 3.201993168252959e-05
BranchID : -0.013476184579342963 , 1.280498475571194e-06
Tenure : 0.034244640415764485 , 8.092889779755567e-35
AssetCost : 0.052199276896215854 , 1.3262665742415235e-78
AmountFinance : 0.10012483824993787 , 6.808152135012277e-285
DisbursalAmount : 0.10040034066950385 , 1.8495613850336294e-286
EMI : 0.04597976851852303 , 2.190006623940473e-61
ManufacturerID : 0.01626845824697694 , 5.0262533774349665e-09
SupplierID : -0.019892066396951593 , 8.758458600850656e-13
LTV : 0.08018399315342986 , 3.815192468296865e-183
SEX : 0.010978849778546969 , 7.971250312494788e-05
AGE : -0.006326431575961772 , 0.02300419363941589
MonthlyIncome : -2.4550687593860024e-05 , 0.9929611113820958
State : -0.058227550439505844 , 2.253

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [161]:
df_merge_train['Top-up Month'].value_counts()

0    107030
6      8398
5      3701
3      3507
4      3075
2      2374
1      1043
Name: Top-up Month, dtype: int64

In [182]:
df0 = df_merge_train[df_merge_train['Top-up Month'] == 0]
df1 = df_merge_train[df_merge_train['Top-up Month'] == 1]
df2 = df_merge_train[df_merge_train['Top-up Month'] == 2]
df3 = df_merge_train[df_merge_train['Top-up Month'] == 3]
df4 = df_merge_train[df_merge_train['Top-up Month'] == 4]
df5 = df_merge_train[df_merge_train['Top-up Month'] == 5]
df6 = df_merge_train[df_merge_train['Top-up Month'] == 6]

df0 = resample(df0, replace=False, n_samples=50000, random_state=150)
df1 = resample(df1, replace=True, n_samples=50000, random_state=150)
df2 = resample(df2, replace=True, n_samples=50000, random_state=150)
df3 = resample(df3, replace=True, n_samples=50000, random_state=150)
df4 = resample(df4, replace=True, n_samples=50000, random_state=150)
df5 = resample(df5, replace=True, n_samples=50000, random_state=150)
df6 = resample(df6, replace=True, n_samples=50000, random_state=150)

df_merge_train1 = pd.concat([df0, df1, df2, df3, df4, df5, df6])
df_merge_train1['Top-up Month'].value_counts()

6    50000
5    50000
4    50000
3    50000
2    50000
1    50000
0    50000
Name: Top-up Month, dtype: int64

## Model Development

In [202]:
Y_train = df_merge_train1['Top-up Month'].values
X_train = df_merge_train1.drop(['ID','Top-up Month'], axis = 1)
X_test = df_merge_test.drop(['ID'], axis = 1)

print(X_train.shape, Y_train.shape)
print(X_test.shape)

(350000, 29) (350000,)
(14745, 29)


In [203]:
from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit(X_train).transform(X_train)
X_test = StandardScaler().fit(X_test).transform(X_test)

In [204]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (280000, 29) (280000,)
Test set: (70000, 29) (70000,)


In [212]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 100)
RFC.fit(x_train, y_train)

RFC_train_predict = RFC.predict(x_train)
RFC_test_predict = RFC.predict(x_test)

print('Train set Accuracy', accuracy_score(y_train, RFC_train_predict))
print('Test set Accuracy', accuracy_score(y_test, RFC_test_predict))
print(classification_report(y_test, RFC_test_predict))

Train set Accuracy 1.0
Test set Accuracy 0.9844714285714286
              precision    recall  f1-score   support

           0       0.99      0.90      0.94     10103
           1       1.00      1.00      1.00      9897
           2       0.99      1.00      0.99      9834
           3       0.98      1.00      0.99      9985
           4       0.99      1.00      0.99     10102
           5       0.99      1.00      1.00     10174
           6       0.96      0.99      0.98      9905

    accuracy                           0.98     70000
   macro avg       0.98      0.98      0.98     70000
weighted avg       0.98      0.98      0.98     70000



In [187]:
from sklearn.ensemble import AdaBoostClassifier
Adaboost = AdaBoostClassifier(n_estimators = 200, learning_rate = 0.5, random_state=0)
Adaboost.fit(x_train, y_train)

Adaboost_train_predict = Adaboost.predict(x_train)
Adaboost_test_predict = Adaboost.predict(x_test)

print('Train set Accuracy', accuracy_score(y_train, Adaboost_train_predict))
print('Test set Accuracy', accuracy_score(y_test, Adaboost_test_predict))
print(classification_report(y_test, Adaboost_test_predict))

Train set Accuracy 0.38020714285714285
Test set Accuracy 0.3744142857142857
              precision    recall  f1-score   support

           0       0.32      0.15      0.21     10103
           1       0.36      0.36      0.36      9897
           2       0.37      0.30      0.33      9834
           3       0.38      0.46      0.42      9985
           4       0.39      0.40      0.40     10102
           5       0.34      0.26      0.30     10174
           6       0.40      0.69      0.51      9905

    accuracy                           0.37     70000
   macro avg       0.37      0.38      0.36     70000
weighted avg       0.37      0.37      0.36     70000



In [205]:
from lightgbm import LGBMClassifier
Lgbm = LGBMClassifier(n_estimators=100, num_leaves=31, max_depth=15, random_state =0)
Lgbm.fit(x_train, y_train)

Lgbm_train_predict = Lgbm.predict(x_train)
Lgbm_test_predict = Lgbm.predict(x_test)

print('Train set Accuracy', accuracy_score(y_train, Lgbm_train_predict))
print('Test set Accuracy', accuracy_score(y_test, Lgbm_test_predict))
print(classification_report(y_test, Lgbm_test_predict))

Train set Accuracy 0.6052428571428572
Test set Accuracy 0.5875142857142858
              precision    recall  f1-score   support

           0       0.50      0.29      0.37     10103
           1       0.74      0.79      0.77      9897
           2       0.62      0.69      0.65      9834
           3       0.62      0.57      0.59      9985
           4       0.63      0.52      0.57     10102
           5       0.57      0.48      0.52     10174
           6       0.47      0.78      0.58      9905

    accuracy                           0.59     70000
   macro avg       0.59      0.59      0.58     70000
weighted avg       0.59      0.59      0.58     70000



In [189]:
from xgboost.sklearn import XGBClassifier
XGB = XGBClassifier(n_estimators=100, learning_rate=0.5, max_depth=15, random_state=0)
XGB.fit(x_train, y_train)

XGB_train_predict = XGB.predict(x_train)
XGB_test_predict = XGB.predict(x_test)

print('Train set Accuracy', accuracy_score(y_train, XGB_train_predict))
print('Test set Accuracy', accuracy_score(y_test, XGB_test_predict))
print(classification_report(y_test, XGB_test_predict))

Train set Accuracy 1.0
Test set Accuracy 0.9771
              precision    recall  f1-score   support

           0       0.99      0.85      0.91     10103
           1       0.99      1.00      1.00      9897
           2       0.98      1.00      0.99      9834
           3       0.98      1.00      0.99      9985
           4       0.99      1.00      0.99     10102
           5       0.98      1.00      0.99     10174
           6       0.93      0.99      0.96      9905

    accuracy                           0.98     70000
   macro avg       0.98      0.98      0.98     70000
weighted avg       0.98      0.98      0.98     70000



In [213]:
prediction = RFC.predict(X_test)
print(len(prediction))
set(prediction)

14745


{0, 1, 2, 3, 4, 5}

### Ensemble

In [191]:
prediction_train1 = RFC.predict(x_train)
prediction_test1 = RFC.predict(X_test)

prediction_train2 = Adaboost.predict(x_train)
prediction_test2 = Adaboost.predict(X_test)

prediction_train3 = Lgbm.predict(x_train)
prediction_test3 = Lgbm.predict(X_test)

prediction_train4 = XGB.predict(x_train)
prediction_test4 = XGB.predict(X_test)

In [192]:
x_train = pd.DataFrame({'A':prediction_train1,'B':prediction_train2,'C':prediction_train3,'D':prediction_train4})
X_test = pd.DataFrame({'A':prediction_test1,'B':prediction_test2,'C':prediction_test3,'D':prediction_test4})

In [193]:
from sklearn import svm
SVM = svm.SVC(kernel='rbf', gamma = 'auto')
SVM.fit(x_train, y_train)

print(classification_report(y_train, SVM.predict(x_train)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39897
           1       1.00      1.00      1.00     40103
           2       1.00      1.00      1.00     40166
           3       1.00      1.00      1.00     40015
           4       1.00      1.00      1.00     39898
           5       1.00      1.00      1.00     39826
           6       1.00      1.00      1.00     40095

    accuracy                           1.00    280000
   macro avg       1.00      1.00      1.00    280000
weighted avg       1.00      1.00      1.00    280000



In [194]:
prediction = SVM.predict(X_test)
print(len(prediction))
set(prediction)

14745


{0, 1, 2, 3, 4, 6}

## Submission

In [214]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LTFS Data Science FinHack 3/sample_submission_ejm25Dc.csv')
print(sample_submission.shape)
sample_submission.head()

(14745, 2)


,ID,Top-up Month
0,4,36-48 Months
1,5,18-24 Months
2,6,30-36 Months
3,25,30-36 Months
4,119,30-36 Months


In [215]:
l = list(prediction)
d = {0:'No Top-up Service',1:'12-18 Months',2:'18-24 Months',3:'24-30 Months',4:'30-36 Months',5:'36-48 Months',6:' > 48 Months'}
for i in range(len(l)):
  l[i] = d[l[i]]
sample_submission['Top-up Month'] = l

In [216]:
sample_submission.to_csv('prediction.csv', index=False)
sample_submission.head()

,ID,Top-up Month
0,4,No Top-up Service
1,5,No Top-up Service
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service


In [217]:
sample_submission['Top-up Month'].value_counts()

No Top-up Service    14476
24-30 Months           139
18-24 Months            89
30-36 Months            25
12-18 Months            14
36-48 Months             2
Name: Top-up Month, dtype: int64